In [1]:
import pybullet as p
import pybullet_data
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import SAC 
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecMonitor

import time
from stable_baselines3.common.callbacks import ProgressBarCallback

pybullet build time: Jan 29 2025 23:16:28


In [2]:
class MazeCarEnv(gym.Env):
    metadata = {'render_modes': ['human'], "render_fps": 500}

    def __init__(self, render_mode=None):
        super().__init__()

        # --- Define Action Space ---
        # Continuous actions: [left_motor_speed, right_motor_speed] ∈ [-1.0, 1.0]
        self.action_space = spaces.Box(
            low=np.array([0.0, 0.0], dtype=np.float32),
            high=np.array([1.0, 1.0], dtype=np.float32),
            dtype=np.float32
        )
        
        # --- Define Observation Space ---
        # Beobachtung: [Auto_x, Auto_y, Auto_yaw, Ziel_x, Ziel_y]
        low = np.array([-6, -6, -np.pi, -6, -6], dtype=np.float32)
        high = np.array([6, 6, np.pi, 6, 6], dtype=np.float32)
        self.observation_space = spaces.Dict({
            "state": spaces.Box(
                low=np.array([-6, -6, -np.pi, -6, -6], dtype=np.float32),
                high=np.array([6, 6, np.pi, 6, 6], dtype=np.float32),
                dtype=np.float32
            ),
            "camera": spaces.Box(
                low=0,
                high=255,
                shape=(64, 64, 4),  # RGBA Bild vom PyBullet
                dtype=np.uint8
            )
        })

        # --- PyBullet Setup ---
        self.render_mode = render_mode
        self.client = p.connect(p.DIRECT if render_mode is None else p.GUI)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81, physicsClientId=self.client)
        p.setTimeStep(1.0 / 240.0, physicsClientId=self.client)  # Standard 240 Hz

        self.planeId = p.loadURDF("plane.urdf", physicsClientId=self.client)

        self.mazeId = p.loadURDF("urdf/maze_colored.urdf",
                                 basePosition=[0, 0, 0],
                                 useFixedBase=True, # keeps maze static
                                 physicsClientId=self.client)


        self.goal_area_1 = np.array([-5.5, 4.5])   # oberer Ausgang
        self.goal_area_2 = np.array([5.5, -4.5])  # unterer Ausgang
        self.goal_radius = 0.5
        self.target_goal_pos = None  # Wird in reset() gesetzt
        self.correct_goal_index = None  # Zufällige Wahl des Ziels

        self.goal_spheres = []

        # Roboter (Auto) laden
        self.start_pos = [-1.0, -2.0, 0.1] 
        self.start_orn = p.getQuaternionFromEuler([0, 0, 0])
        self.carId = p.loadURDF("urdf/simple_two_wheel_car.urdf",
                                self.start_pos, self.start_orn,
                                physicsClientId=self.client)
        
        self.left_wheel_joint_index = 1
        self.right_wheel_joint_index = 0

        self.step_counter = 0
        self.max_steps_per_episode = 480

        self.action_repeat = 50

        self.trajectory = []
        self.all_trajectories = []


    def _get_obs(self):
        pos, orn_quat = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        euler = p.getEulerFromQuaternion(orn_quat)
        yaw = euler[2]

        camera_image = self._get_camera_image()

        return {
            "state": np.array([pos[0], pos[1], yaw, self.target_goal_pos[0], self.target_goal_pos[1]], dtype=np.float32),
            "camera": camera_image
        }


    def _get_info(self):
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        dist_goal1 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_1)
        dist_goal2 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_2)

        return {
            "distance_goal1": dist_goal1,
            "distance_goal2": dist_goal2,
            "target_goal_index": self.correct_goal_index
        }

    def reset(self, seed=None, options=None, custom_goal_pos=None):
        super().reset(seed=seed)
        self.step_counter = 0

        # Auto zurücksetzen
        start_x = self.start_pos[0] + self.np_random.uniform(-0.3, 0.3)
        start_y = self.start_pos[1] + self.np_random.uniform(-0.3, 0.3)
        start_yaw = self.np_random.uniform(-np.pi/6, np.pi/6)
        start_orn = p.getQuaternionFromEuler([0, 0, start_yaw])
        p.resetBasePositionAndOrientation(self.carId, [start_x, start_y, self.start_pos[2]], start_orn, physicsClientId=self.client)
        p.resetBaseVelocity(self.carId,
                            linearVelocity=[0, 0, 0],
                            angularVelocity=[0, 0, 0],
                            physicsClientId=self.client)

        # Entferne bestehende Ziel-Sphären
        for sphere_id in self.goal_spheres:
            p.removeBody(sphere_id, physicsClientId=self.client)
        self.goal_spheres.clear()


        goal_visual_shape = p.createVisualShape(
            p.GEOM_SPHERE,
            radius=self.goal_radius,
            rgbaColor=[0, 1, 0, 0.8]  # Bright green
        )
        
         # Set the goal position
        if custom_goal_pos is not None:
            self.target_goal_pos = np.array(custom_goal_pos)
            self.correct_goal_index = -1  # Custom goal, no correct index
        else:
            self.correct_goal_index = self.np_random.integers(0, 2)
            self.target_goal_pos = self.goal_area_1 if self.correct_goal_index == 0 else self.goal_area_2
        

        # if self.correct_goal_index == 0:
        #     self.target_goal_pos = self.goal_area_1
            
        #     if self.render_mode == "human":
        #         goal_id = p.createMultiBody(
        #             baseVisualShapeIndex=goal_visual_shape,
        #             basePosition=[self.goal_area_1[0], self.goal_area_1[1], 0.1],
        #             useMaximalCoordinates=True
        #         )
        #         self.goal_spheres.append(goal_id)
        # else:
        #     self.target_goal_pos = self.goal_area_2
            
        #     if self.render_mode == "human":
        #         goal_id = p.createMultiBody(
        #             baseVisualShapeIndex=goal_visual_shape,
        #             basePosition=[self.goal_area_2[0], self.goal_area_2[1], 0.1],
        #             useMaximalCoordinates=True
        #         )
        #         self.goal_spheres.append(goal_id)

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()
        
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        self.prev_dist_to_goal = np.linalg.norm(np.array(car_pos[:2]) - self.target_goal_pos)

        if len(self.trajectory) > 0:
            self.all_trajectories.append(self.trajectory)  # Save old episode

        self.trajectory = []
        self.trajectory.append(car_pos[:2])  # only x, y

        return observation, info


    def step(self, action):

        # Clip the action to make sure it's in [-1, 1] range
        action = np.clip(action, self.action_space.low, self.action_space.high)

        # Scale action to desired motor speeds
        max_motor_speed = 10.0  # you can adjust this if needed
        left_vel = float(action[0] * max_motor_speed)
        right_vel = float(action[1] * max_motor_speed)

        # left_vel += np.random.normal(0, 0.2)
        # right_vel += np.random.normal(0, 0.2)



        p.setJointMotorControl2(
            bodyUniqueId=self.carId,
            jointIndex=self.left_wheel_joint_index,
            controlMode=p.VELOCITY_CONTROL,
            targetVelocity=left_vel,
            force=20.0
        )
        p.setJointMotorControl2(
            bodyUniqueId=self.carId,
            jointIndex=self.right_wheel_joint_index,
            controlMode=p.VELOCITY_CONTROL,
            targetVelocity=right_vel,
            force=20.0
        )

        # Jetzt die Simulation mehrmals updaten,
        for _ in range(self.action_repeat):
            p.stepSimulation()
        
        self.step_counter += 1

        # Beobachtung + Reward + Done bestimmen
        observation = self._get_obs()
        info = self._get_info()

        terminated = False

        reward = 0.0
        reward -= 0.005

        # Calculate distance to the goal
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        curr_dist_to_goal = np.linalg.norm(np.array(car_pos[:2]) - self.target_goal_pos)

        # Reward for getting closer to the goal
        reward = 1 * (self.prev_dist_to_goal - curr_dist_to_goal)  # Positive reward for reducing distance
        self.prev_dist_to_goal = curr_dist_to_goal

        # Check if the car is in the goal area
        in_goal_1 = np.linalg.norm(car_pos[:2] - self.goal_area_1) < self.goal_radius
        in_goal_2 = np.linalg.norm(car_pos[:2] - self.goal_area_2) < self.goal_radius

        in_goal = np.linalg.norm(car_pos[:2] - self.target_goal_pos) < self.goal_radius

        if in_goal_1:
            if self.correct_goal_index == 0:
                reward = 10.0
                terminated = True
            else:
                reward = -5.0
                terminated = True
        elif in_goal_2:
            if self.correct_goal_index == 1:
                reward = 10.0
                terminated = True
            else:
                reward = -5.0
                terminated = True
        elif in_goal:
            if self.correct_goal_index == -1:
                reward = 10.0
                terminated = True
            else:
                reward = -5.0
                terminated = True

        # print("Reward: ", reward)
        # print(self.prev_dist_to_goal)

        self.trajectory.append(car_pos[:2])

        contacts = p.getContactPoints(bodyA=self.carId, bodyB=self.mazeId, physicsClientId=self.client)
        if len(contacts) > 0:
            reward -= 3.0
            terminated = True 

        truncated = (self.step_counter >= self.max_steps_per_episode)

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, truncated, info

    def render(self):
        # Bei PyBullet im GUI-Modus passiert das Rendering automatisch
        pass

    def _render_frame(self):
        # Debug-Anzeigen, falls erwünscht
        pass

    def close(self):
        p.disconnect(physicsClientId=self.client)

    def _get_camera_image(self):
        car_pos, car_orn = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        car_euler = p.getEulerFromQuaternion(car_orn)

        camera_distance = 0.1
        camera_height = 0.2
        camera_yaw = car_euler[2] * 180 / np.pi
        target_pos = [
            car_pos[0] + camera_distance * np.cos(car_euler[2]),
            car_pos[1] + camera_distance * np.sin(car_euler[2]),
            car_pos[2] + camera_height
        ]

        width, height, rgba, _, _ = p.getCameraImage(
            width=64,
            height=64,
            viewMatrix=p.computeViewMatrix(
                cameraEyePosition=[car_pos[0], car_pos[1], car_pos[2] + camera_height],
                cameraTargetPosition=target_pos,
                cameraUpVector=[0, 0, 1]
            ),
            projectionMatrix=p.computeProjectionMatrixFOV(
                fov=70,
                aspect=1.0,
                nearVal=0.01,
                farVal=10.0
            ),
            physicsClientId=self.client
        )

        rgba_image = np.reshape(rgba, (height, width, 4)).astype(np.uint8)
        return rgba_image



In [3]:
def make_env(render_mode=None):
    def _init():
        return MazeCarEnv(render_mode=render_mode)
    return _init

num_envs = 16

In [4]:
# env = MazeCarEnv(render_mode=None) 
env = MazeCarEnv(render_mode="human")

p.resetDebugVisualizerCamera(cameraDistance=9, cameraYaw=-92, cameraPitch=-85, cameraTargetPosition=[0, 0, 0])

try:
    check_env(env)
    print("Environment check passed!")
    env.close()
except Exception as e:
    print(f"Environment check failed: {e}")
    env.close()
    exit()

env = SubprocVecEnv([make_env(render_mode=None) for _ in range(num_envs)])

env = VecMonitor(env, "./ppo_mazecar_tensorboard/")  # Log to the specified directory

model = SAC("MultiInputPolicy", 
            env, 
            ent_coef="auto",
            gamma=0.96,
            # verbose=1, 
            tensorboard_log="./sac_mazecar_tensorboard/",
            batch_size=256, # after n_steps the data is split into batches
            learning_rate=0.0003,
            train_freq=(512, "step"),  # how often to train
            gradient_steps=512,        # steps per update
            buffer_size=1_000_000,     # Wichtiger, damit nichts verloren geht
            device="cuda"
)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-P)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
Environment chec

/home/ferdinand/masterthesis/ve_pybullet/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:462: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
/home/ferdinand/masterthe

In [ ]:
# eval_env = MazeCarEnv(render_mode="human")
# eval_freq = 2000
# eval_callback = EvalCallback(eval_env, eval_freq=eval_freq, verbose=1)

# 4. Start training
model.learn(total_timesteps=1_000_000, progress_bar=True)


model.save("models/sac_mazecar_model_2")

env.close()
# check_env.close()

Output()

In [ ]:
np.save("all_trajectories.npy", np.array(env.get_attr("all_trajectories")[0], dtype=object))


import matplotlib.pyplot as plt

all_trajectories = np.load("all_trajectories.npy", allow_pickle=True)

for ep_traj in all_trajectories:
    plt.plot(np.array(ep_traj)[:,0], np.array(ep_traj)[:,1])

plt.xlabel("X")
plt.ylabel("Y")
plt.title("All Trajectories over Training")
plt.grid()
plt.show()


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
# --- To evaluate ---
custom_goal_pos = [-5.5, -4.5]  # Example custom goal position

eval_env = MazeCarEnv(render_mode="human")  # Render during evaluation
model = SAC.load("models/sac_mazecar_model_straight_loop_250000", env=eval_env)
# model = PPO.load("models/ppo_mazecar_model_2", env=eval_env)

# Set the camera to the same position as for training
p.resetDebugVisualizerCamera(cameraDistance=9, cameraYaw=-92, cameraPitch=-85, cameraTargetPosition=[0, 0, 0])


print(".......................")
obs, info = eval_env.reset(custom_goal_pos=None)
print(".......................")

for _ in range(800):  # Max steps for evaluation
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = eval_env.step(action)
    time.sleep(1./240.0)  # Adjust sleep time for rendering speed
    if terminated or truncated:
        print(f"Eval episode finished, Reached goal: {info.get('target_goal_index')}, Reward: {reward}")
        obs, info = eval_env.reset(custom_goal_pos=None)  # Reset for next evaluation episode

eval_env.close()

/home/ferdinand/masterthesis/ve_pybullet/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:606: 
UserWarning: This system does not have apparently enough memory to store the complete replay buffer 32.82GB > 
2.34GB
  warnings.warn(

.......................

.......................

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-P)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu


In [ ]:
# Create a new environment instance
env = SubprocVecEnv([make_env(render_mode=None) for _ in range(num_envs)])
env = VecMonitor(env, "./sac_mazecar_tensorboard/")  # Log to the specified directory

# Load the model with the new environment
model = SAC.load("models/sac_mazecar_model", env=env)

model.set_env(env)
model.learn(
    total_timesteps=100_000,
    reset_num_timesteps=False,
    tb_log_name="SAC_9",  # wichtig: GLEICHER Name!
    progress_bar=True
)

# Continue training the model
# model.learn(total_timesteps=100_000, progress_bar=True)

# Save the updated model
model.save("models/sac_mazecar_model")

pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28


pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28


pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28


pybullet build time: Jan 29 2025 23:16:28


pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28


FileNotFoundError: [Errno 2] No such file or directory: 'models/sac_mazecar_model.zip'

pybullet build time: Jan 29 2025 23:16:28
pybullet build time: Jan 29 2025 23:16:28


In [ ]:
env.close()

check_env.close()

# car_pos, _ = p.getBasePositionAndOrientation(eval_env.carId, physicsClientId=eval_env.client)
# p.resetDebugVisualizerCamera(cameraDistance=3, cameraYaw=50, cameraPitch=-35, cameraTargetPosition=car_pos)

EOFError: 

In [ ]:
eval_env.close()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [ ]:
# ~/masterthesis/mt_start$   tensorboard --logdir=./ppo_mazecar_tensorboard/

In [ ]:
import torch
print(torch.cuda.is_available())  # Should return True if GPU is available
print(torch.cuda.get_device_name(0))  # Prints the name of the GPU

True
NVIDIA GeForce RTX 4060 Laptop GPU
